# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [59]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from statistics import mean
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code

200

In [8]:
data = data.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2261884292901,
 'week52high': 142.93,
 'week52low': 56.33,
 'week52change': 0.805057510247024,
 'sharesOutstanding': 17668591470,
 'float': 0,
 'avg10Volume': 111289644,
 'avg30Volume': 110658977,
 'day200MovingAvg': 117.8,
 'day50MovingAvg': 128.65,
 'employees': 0,
 'ttmEPS': 3.43,
 'ttmDividendRate': 0.8159980172117699,
 'dividendYield': 0.006324424869866673,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-27',
 'nextEarningsDate': '0',
 'peRatio': 40.095085378566196,
 'beta': 1.1711382557854413,
 'maxChangePercent': 52.148868356643035,
 'year5ChangePercent': 4.656026101587502,
 'year2ChangePercent': 2.4784513030333333,
 'year1ChangePercent': 0.7887468380352926,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.4631345919105152,
 'month3ChangePercent': 0.1803244187596042,
 'month1ChangePercent': 0.08108085187933463,
 'day30ChangePercent': 0.08093920588206753,
 'day5ChangePercent': 0.005596714074966109}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [9]:
data['year1ChangePercent']

0.7887468380352926

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [16]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

col = ['Ticker', 'Price', 'One-Year Price Return', 'Purchase Amount']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [17]:
main_df = pd.DataFrame(columns=col)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        main_df = main_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index= col
            ),
            ignore_index=True
        )

In [18]:
main_df

,Ticker,Price,One-Year Price Return,Purchase Amount
0,A,121.50,0.397303,N/A
1,AAL,15.97,-0.469897,N/A
2,AAP,160.92,-0.00423156,N/A
3,AAPL,134.30,0.794369,N/A
4,ABBV,111.54,0.26311,N/A
...,...,...,...,...
500,YUM,111.61,0.0842464,N/A
501,ZBH,156.46,0.0411783,N/A
502,ZBRA,390.09,0.500057,N/A
503,ZION,43.61,-0.145188,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [85]:

final_df = main_df.sort_values(by= 'One-Year Price Return', ascending= False)
final_df = final_df[:50]
final_df.reset_index(inplace=True, drop= True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [86]:
def portfolio_input():
    global portfolio_size

    while True:
        try:
            portfolio_size = float(input('Enter the value of your portfolio:'))

        except ValueError:
            print ("Oops!  That was not a valid number.  Try again...")
            
        else:
            break

portfolio_input()

In [87]:
position_size = portfolio_size/len(final_df.index)
for i in range (0, len(final_df.index)):
    final_df.loc[i,'Purchase Amount'] = math.floor(position_size/final_df.loc[i,'Price'])

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [111]:
hqm_col =[
    'Ticker',
    'Price',
    'Purchase Amount',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_col)

In [112]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index= hqm_col
            ),
            ignore_index=True
        )

In [113]:
hqm_df

,Ticker,Price,Purchase Amount,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.70,N/A,0.405954,N/A,0.34586,N/A,0.194485,N/A,0.0348851,N/A,N/A
1,AAL,16.38,N/A,-0.46372,N/A,0.270507,N/A,0.213429,N/A,0.0629006,N/A,N/A
2,AAP,159.59,N/A,-0.00432159,N/A,0.115873,N/A,0.0148957,N/A,0.0643469,N/A,N/A
3,AAPL,136.25,N/A,0.791052,N/A,0.483705,N/A,0.178105,N/A,0.0814007,N/A,N/A
4,ABBV,110.93,N/A,0.270837,N/A,0.112085,N/A,0.264374,N/A,0.0210429,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.96,N/A,0.0855899,N/A,0.274946,N/A,0.147093,N/A,0.0349156,N/A,N/A
501,ZBH,158.14,N/A,0.0411201,N/A,0.286409,N/A,0.123866,N/A,0.0456383,N/A,N/A
502,ZBRA,396.37,N/A,0.499624,N/A,0.513155,N/A,0.507445,N/A,0.0487521,N/A,N/A
503,ZION,45.44,N/A,-0.141637,N/A,0.368727,N/A,0.469672,N/A,0.0445904,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [127]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

# Kept getting TypeError: '<' not supported between instances of 'NoneType' and 'float'
# So I needed to work around this a mutilate anones to 0
for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_df.loc[row, change_col] == None:
            hqm_df.loc[row, change_col] = 0.0

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, percentile_col] = score(hqm_df[change_col], hqm_df.loc[row,change_col],)/100

hqm_df

,Ticker,Price,Purchase Amount,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.70,N/A,0.405954,0.871287,0.34586,0.685149,0.194485,0.588119,0.0348851,0.576238,23.8614
1,AAL,16.38,N/A,-0.46372,0.019802,0.270507,0.534653,0.213429,0.627723,0.0629006,0.778218,23.3663
2,AAP,159.59,N/A,-0.00432159,0.352475,0.115873,0.275248,0.0148957,0.176238,0.0643469,0.784158,23.4158
3,AAPL,136.25,N/A,0.791052,0.970297,0.483705,0.843564,0.178105,0.538614,0.0814007,0.851485,24.2574
4,ABBV,110.93,N/A,0.270837,0.754455,0.112085,0.269307,0.264374,0.706931,0.0210429,0.483168,23.8614
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.96,N/A,0.0855899,0.49901,0.274946,0.550495,0.147093,0.467327,0.0349156,0.578218,23.6139
501,ZBH,158.14,N/A,0.0411201,0.427723,0.286409,0.582178,0.123866,0.421782,0.0456383,0.667327,23.6139
502,ZBRA,396.37,N/A,0.499624,0.916832,0.513155,0.885149,0.507445,0.956436,0.0487521,0.689109,24.1584
503,ZION,45.44,N/A,-0.141637,0.180198,0.368727,0.718812,0.469672,0.936634,0.0445904,0.659406,23.5149


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [128]:
for row in hqm_df.index:
    momentum_percentiles = []

    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile']) 
          
    hqm_df.loc[row,'HQM Score'] = mean(momentum_percentiles)    

In [129]:
hqm_df

,Ticker,Price,Purchase Amount,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,121.70,N/A,0.405954,0.871287,0.34586,0.685149,0.194485,0.588119,0.0348851,0.576238,0.680198
1,AAL,16.38,N/A,-0.46372,0.019802,0.270507,0.534653,0.213429,0.627723,0.0629006,0.778218,0.490099
2,AAP,159.59,N/A,-0.00432159,0.352475,0.115873,0.275248,0.0148957,0.176238,0.0643469,0.784158,0.39703
3,AAPL,136.25,N/A,0.791052,0.970297,0.483705,0.843564,0.178105,0.538614,0.0814007,0.851485,0.80099
4,ABBV,110.93,N/A,0.270837,0.754455,0.112085,0.269307,0.264374,0.706931,0.0210429,0.483168,0.553465
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.96,N/A,0.0855899,0.49901,0.274946,0.550495,0.147093,0.467327,0.0349156,0.578218,0.523762
501,ZBH,158.14,N/A,0.0411201,0.427723,0.286409,0.582178,0.123866,0.421782,0.0456383,0.667327,0.524752
502,ZBRA,396.37,N/A,0.499624,0.916832,0.513155,0.885149,0.507445,0.956436,0.0487521,0.689109,0.861881
503,ZION,45.44,N/A,-0.141637,0.180198,0.368727,0.718812,0.469672,0.936634,0.0445904,0.659406,0.623762


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [130]:
final_hqm_df = hqm_df.sort_values(by= 'HQM Score', ascending= False)
final_hqm_df = final_hqm_df[:50]
final_hqm_df.reset_index(inplace=True, drop= True)

In [131]:
final_hqm_df

,Ticker,Price,Purchase Amount,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,149.46,N/A,1.10632,0.992079,0.923692,0.986139,0.674597,0.984158,0.0959465,0.908911,0.967822
1,FCX,26.62,N/A,0.979044,0.990099,1.27186,0.99604,0.658526,0.980198,0.0826784,0.859406,0.956436
2,SIVB,390.10,N/A,0.525545,0.926733,0.895727,0.982178,0.601817,0.970297,0.0998295,0.914851,0.948515
3,ALGN,553.61,N/A,0.900303,0.980198,0.948031,0.990099,0.72157,0.992079,0.0587143,0.758416,0.930198
4,MU,78.51,N/A,0.35835,0.833663,0.511797,0.881188,0.626188,0.974257,0.0921637,0.90297,0.89802
5,DIS,188.26,N/A,0.223761,0.69505,0.62096,0.934653,0.499964,0.946535,0.184947,0.988119,0.891089
6,ALXN,157.65,N/A,0.464103,0.90495,0.393852,0.754455,0.409367,0.89901,0.280781,0.99802,0.889109
7,TTWO,208.23,N/A,0.710262,0.958416,0.455896,0.821782,0.309316,0.80396,0.148125,0.964356,0.887129
8,APTV,135.01,N/A,0.358036,0.831683,0.712167,0.962376,0.402256,0.893069,0.0802235,0.843564,0.882673
9,CDNS,142.35,N/A,0.915972,0.984158,0.405002,0.764356,0.304966,0.792079,0.177799,0.982178,0.880693


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [119]:
portfolio_input()

In [132]:
position_size = portfolio_size/len(final_hqm_df.index)
for i in range (0, len(final_hqm_df.index)):
    final_hqm_df.loc[i,'Purchase Amount'] = math.floor(position_size/final_hqm_df.loc[i,'Price'])

In [133]:
final_hqm_df

,Ticker,Price,Purchase Amount,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,149.46,133,1.10632,0.992079,0.923692,0.986139,0.674597,0.984158,0.0959465,0.908911,0.967822
1,FCX,26.62,751,0.979044,0.990099,1.27186,0.99604,0.658526,0.980198,0.0826784,0.859406,0.956436
2,SIVB,390.10,51,0.525545,0.926733,0.895727,0.982178,0.601817,0.970297,0.0998295,0.914851,0.948515
3,ALGN,553.61,36,0.900303,0.980198,0.948031,0.990099,0.72157,0.992079,0.0587143,0.758416,0.930198
4,MU,78.51,254,0.35835,0.833663,0.511797,0.881188,0.626188,0.974257,0.0921637,0.90297,0.89802
5,DIS,188.26,106,0.223761,0.69505,0.62096,0.934653,0.499964,0.946535,0.184947,0.988119,0.891089
6,ALXN,157.65,126,0.464103,0.90495,0.393852,0.754455,0.409367,0.89901,0.280781,0.99802,0.889109
7,TTWO,208.23,96,0.710262,0.958416,0.455896,0.821782,0.309316,0.80396,0.148125,0.964356,0.887129
8,APTV,135.01,148,0.358036,0.831683,0.712167,0.962376,0.402256,0.893069,0.0802235,0.843564,0.882673
9,CDNS,142.35,140,0.915972,0.984158,0.405002,0.764356,0.304966,0.792079,0.177799,0.982178,0.880693


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [139]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
final_hqm_df.to_excel(writer, sheet_name='Momentum Strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [140]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [141]:
column_format = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Purchase Amount',integer_template],
    'D':['One-Year Price Return',percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',18, column_format[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_format[column][0],column_format[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [142]:
writer.save()